In [1]:
# pip install chardet
# pip install pyphen

In [2]:
## Importing the necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

import os
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pyphen
import chardet

In [3]:
df = pd.read_excel("input.xlsx") ## loading the csv file

In [4]:
df

URL_ID                                                URL
0      123.0  https://insights.blackcoffer.com/rise-of-telem...
1      321.0  https://insights.blackcoffer.com/rise-of-e-hea...
2     2345.0  https://insights.blackcoffer.com/rise-of-e-hea...
3     4321.0  https://insights.blackcoffer.com/rise-of-telem...
4      432.0  https://insights.blackcoffer.com/rise-of-telem...
..       ...                                                ...
109  50921.0  https://insights.blackcoffer.com/coronavirus-i...
110  51382.8  https://insights.blackcoffer.com/coronavirus-i...
111  51844.6  https://insights.blackcoffer.com/what-are-the-...
112  52306.4  https://insights.blackcoffer.com/marketing-dri...
113  52768.2  https://insights.blackcoffer.com/continued-dem...

[114 rows x 2 columns]

In [5]:
rows=df.shape[0]  ## checking the number of rows in the file
rows

114

#### Extracting the data from the URL and storing it in the text file

In [6]:
for i in range(0,rows):
    url = df.URL[i]
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Check for different class names for titles
        title_element = soup.find('h1', class_='entry-title') or soup.find('h1', class_='tdb-title-text')

        if title_element:
            title = title_element.text.strip()

            # Find the article content
            article = soup.find('div', class_='td-post-content tagdiv-type') or soup.find('div', class_='tdb-block-inner td-fix-index')

            if article:
                paragraphs = article.find_all('p')
                all_text = '\n'.join([p.get_text() for p in paragraphs])

                # Clean and encode the article text
                all_text = all_text.replace('\n', ' ').replace('#', ' ')
                article_encoded = all_text.encode('utf-8')

                   
                file_path = os.path.join('textfiles', f'{df.URL_ID[i]}.txt')
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(title + '\n')
                    f.write(article_encoded.decode('utf-8'))

        else:
            print(f"Title not found for URL {url}")

    else:
        print(f"Failed to retrieve URL {url}")


Failed to retrieve URL https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to retrieve URL https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


### For these two links the page was not found

11668 -- https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/  
  
17671.4 -- https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/

In [7]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### USING THE TEXT FILES FOR STOP WORDS, POSITITVE AND NEGATIVE WORDS

In [8]:
# Initialize empty lists for stop words, positive words, and negative words

stop_words = []
positive_words = []
negative_words = []

stop_words_dir = 'StopWords' 

# Loop through each text file in the "StopWords" folder

for filename in os.listdir(stop_words_dir):
    if filename.endswith(".txt"):
        with open(os.path.join(stop_words_dir, filename), 'r') as stop_file:
            stop_words += stop_file.read().splitlines()
            
            

# Load positive words from 'positive_words.txt' file
with open('positive-words.txt', 'r') as pos_file:
    positive_words = pos_file.read().splitlines()
    

# Load negative words from 'negative_words.txt' file
with open('negative-words.txt', 'r') as neg_file:
    negative_words = neg_file.read().splitlines()
    


complex_word_threshold = 2


#### COUNTING THE SYLLABLE IN A WORD

In [9]:
def syllable_count(word):
    dic = pyphen.Pyphen(lang='en_US')
    return len(dic.inserted(word).split('-'))

#### GIVING THE LOCATION OF THE TEXT FILES

In [10]:
results = []

# Directory containing the text files
directory = 'C:\\Users\\khara\\Blackcoffer Internship\\TextFiles'

#### PERFORMING OPERATIONS ON THE TEXT FILES ON WHICH THE OPERATION IS TO BE PERFORMED

In [11]:
for filename in os.listdir(directory):
    if filename.endswith(".txt"): 
        with open(os.path.join(directory, filename), 'rb') as file:
            raw_data = file.read()
            encoding = chardet.detect(raw_data)['encoding']
            text = raw_data.decode(encoding)

            # Tokenize the text into sentences and words
            sentences = sent_tokenize(text)
            words = word_tokenize(text)

            # Initialize variables for analysis
            positive_score = 0
            negative_score = 0
            total_word_count = len(words)
            total_sentence_count = len(sentences)
            complex_word_count = 0
            total_syllables = 0
            personal_pronouns = 0
            total_characters = 0

            # Calculate scores and counts
            for word in words:
                word = word.lower()  # Convert to lowercase for word matching
                if word in positive_words:
                    positive_score += 1
                elif word in negative_words:
                    negative_score += 1

                if word not in stop_words:
                    # Calculate syllables for non-stop words using pyphen
                    syllables = syllable_count(word)
                    total_syllables += syllables

                    # Count complex words
                    if syllables >= complex_word_threshold:
                        complex_word_count += 1

                    # Check for personal pronouns
                    if re.match(r'\b(I|we|my|ours|us)\b', word):
                        personal_pronouns += 1

                total_characters += len(word)  # Count characters for average word length

            # Calculate polarity and subjectivity scores
            polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
            subjectivity_score = (positive_score + negative_score) / (total_word_count + 0.000001)

            # Calculate average sentence length
            average_sentence_length = total_word_count / total_sentence_count

            # Calculate percentage of complex words
            percentage_complex_words = (complex_word_count / total_word_count) * 100

            # Calculate Fog Index
            fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

            # Calculate average word length
            average_word_length = total_characters / total_word_count

            # Append the results to the list
            results.append({
                'Filename': filename,
                'Positive Score': positive_score,
                'Negative Score': negative_score,
                'Polarity Score': polarity_score,
                'Subjectivity Score': subjectivity_score,
                'Average Sentence Length': average_sentence_length,
                'Percentage of Complex Words': percentage_complex_words,
                'Fog Index': fog_index,
                'Average Number of Words Per Sentence': average_sentence_length,
                'Complex Word Count': complex_word_count,
                'Word Count': total_word_count,
                'Syllables Per Word': total_syllables / total_word_count,
                'Personal Pronouns': personal_pronouns,
                'Average Word Length': average_word_length
            })


#### STORING THE VALUES IN A CSV FILE

In [13]:
res = pd.DataFrame(results)

# Save the results to a CSV file
res.to_csv('Output.csv', index=False)

# Print the results DataFrame
print(res)

        Filename  Positive Score  Negative Score  Polarity Score  \
0    10282.6.txt              67              25        0.456522   
1    10744.4.txt              56              23        0.417722   
2    11206.2.txt              29              12        0.414634   
3    12129.8.txt              44              13        0.543860   
4      123.0.txt              89              24        0.575221   
..           ...             ...             ...             ...   
107   7973.6.txt              41              26        0.223881   
108   8435.4.txt               2               0        1.000000   
109   8897.2.txt              71              38        0.302752   
110   9359.0.txt              73              38        0.315315   
111   9820.8.txt              80              27        0.495327   

     Subjectivity Score  Average Sentence Length  Percentage of Complex Words  \
0              0.047594                25.103896                    29.384377   
1              0.0555